In [ ]:
import os
import sys
import requests
from datetime import datetime 
from multiprocessing import Pool
import numpy as  np
import sympy as sp
import pandas as pd
from pathlib import Path
import pickle
from tqdm import tqdm
from time import time
from typing import Dict, List, Optional

import matplotlib.pyplot as plt

from mpl_toolkits.mplot3d import Axes3D
from scipy.integrate import odeint
import numpy.linalg as LA
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
plt.rcParams['font.family'] = 'Arial'
from sympy import *

In [ ]:
#Module
def dndt(v,n):
    a=0.01*(v+34)/(1-np.exp(-(v+34)/10))
    b=0.125*np.exp(-(v+44)/25)
    return 4*(a-a*n-b*n)

def dvdt(v,n,h):
    return (-1)*(IK(v,n)+ILek(v)+ILena(v)+IUNaV(v,h)+IKNa(v)+ICa(v))

def dhdt(v,h):
    return (ssih(v)-h)/tau(v)



def IK(v,n):
    return gK*(n**4)*(v-VK)

def ILeak(v):
    return gLeak*(v-VL)

def IUNaV(v,h):
    return gUNaV*(m(v)**3)*(h)*(v-VNa)

def mna():
    return 1.0/(1.0+(32.0/Na4)**(3.0))

def IKNa(v):
    return gKNa*mna()*(v-VK)

def ICa(v):
    return gCa*mca2(v)*(v-VCa)

def ILek(v):
    return gLek*(v-VLK)
def ILena(v):
    return gLena*(v-VLN)
    

    
def am(v):
    return 0.1*(v+33+x)/(1-np.exp(-(v+33+x)/10))

def bm(v):
    return 4*np.exp(-(v+53.7+x)/12)
def m(v):
    return am(v)/(am(v)+bm(v))

def ah(v):
    return 0.07*np.exp(-(v+50+y)/10)
def bh(v):
    return 1/(1+np.exp(-(v+20+y)/10))

def ssih(v):
    return ah(v)/(ah(v)+bh(v))

def tau(v):
    return 1.0/(4.0*(ah(v)+bh(v)))


def mca2(v):
    Z=1.0 / (1.0 + np.exp(-(v+20.0)/9.0))
    return Z**2


#Parameter
gK= 48.19198701
gUNaV=6.104226316
gKNa=9.65743873
gLeak=0.062345227
gLek=gLeak*0.6096
gLena=gLeak*0.3904
gCa=0.391216425
tNa=6638.79306935
x=28.21858435
y=-7.96971366
Na1=6.5
Na2=7.15
Na4=7.8


VNa=55.0
VK=-100.0
VL=-60.95
VCa=120.0

VLK=-100.0
VLN=0.0


def nKnull(v):
    a=0.01*(v+34)/(1-np.exp(-(v+34)/10))
    b=0.125*np.exp(-(v+44)/25)
    return a/(a+b)



def hnull(v):
    return ssih(v)

def Vnull(V,h):
    
    yyyx=IUNaV(V,h)+ILek(V)+ILena(V)+IKNa(V)+ICa(V)
    zzzx=gK*(V-VK)
    w=-1*yyyx/zzzx
    return w**0.25


def V0det(VVV):
    return Vnull(VVV,hnull(VVV))-nKnull(VVV)

def V0det1(VVV):
    return Vnull(VVV,hnull(VVV))-nKnull(VVV)
def V0det2(VVV):
    return Vnull(VVV,Na2,x,y,hnull(VVV))-nKnull(VVV)
def V0det3(VVV):
    return Vnull(VVV,Na3,x,y,hnull(VVV))-nKnull(VVV)
def V0det4(VVV):
    return Vnull(VVV,Na4,x,y,hnull(VVV))-nKnull(VVV)

In [ ]:
#Overlap2
VVfumo=np.linspace(-95,50,1000)
nkfumo=nKnull(VVfumo)
hunavfumo=ssih(VVfumo)

fig = plt.figure(figsize = (8, 8))
ax = fig.add_subplot(111, projection='3d')
Z=np.linspace(-80,60,5)
ax.set_xticks(Z)
Z=np.linspace(0,1,5)
ax.set_yticks(Z)
ax.set_zticks(Z)
ax.set_title("nK and hNa nullplane Overlap", size = 20)
ax.set_xlabel("V", size = 14)
ax.set_ylabel("nK", size = 14)
ax.set_zlabel("hunav", size = 14)
ax.plot(VVfumo, nkfumo, hunavfumo, color = "Black")
#plt.savefig("Overlap2.pdf")

In [ ]:
VV11=np.linspace(-95,60,10000)
VV1=[]
HH1=[]
NN1=[]
for i in tqdm(range(len(VV11))):
    nnnn=Vnull(VV11[i],hnull(VV11[i]))
    if(nnnn>=0):
        VV1.append(VV11[i])
        HH1.append(hnull(VV11[i]))
        NN1.append(nnnn)

In [ ]:
v0 = [-77,  0.004, 0.7]
t=np.arange(0,200, 0.001)
# モジュールの読み込み

from scipy.integrate import odeint
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
def lorenz_func(v, t):
    return [dvdt(v[0],v[1],v[2]),dndt(v[0],v[1]),dhdt(v[0],v[2])]
# 関数の呼び出し
v = odeint(lorenz_func, v0, t)

# 可視化
fig = plt.figure()
ax=fig.add_subplot(111, projection='3d')
ax.plot(v[:, 0], v[:, 1], v[:, 2])
ax.scatter(v0[0],v0[1],v0[2], color="black")

# ラベルなど
plt.title('Lorenz')
plt.grid(True)
print(t)

In [ ]:
v0 = [30, 0.5, 0.06]
t=np.arange(0,20, 0.001)
# モジュールの読み込み

from scipy.integrate import odeint
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
def lorenz_func(v, t):
    return [dvdt(v[0],v[1],v[2]),dndt(v[0],v[1]),dhdt(v[0],v[2])]
# 関数の呼び出し
v1 = odeint(lorenz_func, v0, t)

# 可視化
fig = plt.figure()
ax=fig.add_subplot(111, projection='3d')
ax.plot(v1[:, 0], v1[:, 1], v1[:, 2])
ax.scatter(v0[0],v0[1],v0[2], color="black")

# ラベルなど
plt.title('Lorenz')
plt.grid(True)
print(t)

In [ ]:
VB=np.linspace(-95.1927,-5.1926,20000)
VC=[]
VD=[]
for i in tqdm(range(len(VB))):
    VC.append(V0det(VB[i]))
    VD.append(0)
        
plt.figure(figsize=(5,5))
plt.plot(VB,VC)
plt.plot(VB,VD)

In [ ]:
for i in range(len(VB)-1):
    if(VC[i]*VC[i+1]<0):
        print(VB[i])

In [ ]:
VB=np.linspace(-87.74,-87.72,100000)
VC=[]
VD=[]
for i in tqdm(range(len(VB))):
    VC.append(V0det(VB[i]))
    VD.append(0)
        
plt.figure(figsize=(5,5))
plt.plot(VB,VC)
plt.plot(VB,VD)

In [ ]:
for i in range(len(VB)-1):
    if(VC[i]*VC[i+1]<0):
        print(VB[i])

In [ ]:
VB=np.linspace(-78,-76,100000)
VC=[]
VD=[]
for i in tqdm(range(len(VB))):
    VC.append(V0det(VB[i]))
    VD.append(0)
        
plt.figure(figsize=(5,5))
plt.plot(VB,VC)
plt.plot(VB,VD)

In [ ]:
for i in range(len(VB)-1):
    if(VC[i]*VC[i+1]<0):
        print(VB[i])

In [ ]:
vpsps1=-36.712211000550035
hpsps1=hnull(vpsps1)
npsps1=nKnull(vpsps1)

vpsps2=-76.58226582265823
hpsps2=hnull(vpsps2)
npsps2=nKnull(vpsps2)

vpsps3=-87.72415484154841
hpsps3=hnull(vpsps3)
npsps3=nKnull(vpsps3)

In [ ]:
fig = plt.figure(figsize = (4, 4))
ax = fig.add_subplot(111, projection='3d')

ax.set_title("Na={} (mM)" .format(str(Na4)), size = 20)
ax.set_xlabel("V (mV)", size = 14)
ax.set_ylabel("nK (unitless)", size = 14)
ax.set_zlabel("hUNaV (unitless)", size = 14)

Z=np.linspace(-80,60,3)
ax.set_xticks(Z)
Z=np.linspace(0,1,3)
ax.set_yticks(Z)
ax.set_zticks(Z)
ax.plot(VVfumo, nkfumo, hunavfumo, color = "Black")
ax.plot(v[:,0],v[:,1],v[:,2], color="red")
ax.plot(v1[:,0],v1[:,1],v1[:,2], color="orange")
ax.plot(v[:,0][0],v[:,1][0],v[:,2][0], marker='o',color="red")
ax.plot(v1[:,0][0],v1[:,1][0],v1[:,2][0], marker='o',color="orange")


ax.plot(vpsps1,npsps1,hpsps1, marker='o', color="black")
ax.plot(vpsps2,npsps2,hpsps2, marker='o', color="black")
ax.plot(vpsps3,npsps3,hpsps3, marker='o', color="black")
#ax.scatter(VV,NN,HH, color="purple", s=0.1)
ax.scatter(VV1,NN1,HH1, color="blue", s=0.1)
ax.axes.yaxis.set_visible(False) 
ax.zaxis.set_tick_params(labelsize=0)
ax.yaxis.set_tick_params(labelsize=0)
ax.xaxis.set_tick_params(labelsize=0)
plt.savefig("N4.pdf")